In [1]:
# Import dati dalla prof
from scipy.io import loadmat
import numpy as np
import scipy.linalg as spl
import RisolviSis as RS
import matplotlib.pyplot as plt
dati = loadmat('testC.mat')
A = dati["A"]
b = dati["b"]


### LU

In [2]:
# Le premesse sono che la matrice A sia non singolare, quindi che
# ne esista l'inversa, che A abbia rango massimo e che A abbia
# determinante diverso da 0. Quest'ultima è importante perche' rende
# la soluzione unica: si vuole dimostrare che esiste una matrice di
# permutazione che valida la seguente relazione: PA = LU, con P matrice
# di permutazione, A matrice orginale, U matrice triangolare superiore,
# L matrice triangolare inferiore con diagonale che presenta tutti i termini
# pari a 1.
#
# Per cercare poi una soluzione valida bisogna soddisfare il seguente sistema
# lineare:
# { Lz = Pb
# { Ux = z
def LUSolve(P, L, U, b):
    y, flag = RS.Lsolve(L, np.dot(P.T, b))
    x, flag = RS.Usolve(U, y)
    return x, flag


In [3]:
# Vedo quanto la matrice e' mal condizionata
# così da capire come poter muovermi.
cond = np.linalg.cond(A)
print(cond)

# Risolvo il problema chiamando la funzione
# di scipy.linalg che mi da' gratis la
# fattorizzazione.
P, L, U = spl.lu(A)
sol, flag = LUSolve(P, L, U, b)

# Soluzione
print(sol)

# Di quanto ci discostiamo dalla soluzione esatta?
x_esatta = np.ones_like(b)
err = np.linalg.norm(sol - x_esatta) / np.linalg.norm(x_esatta)
print("Errore soluzione LU: ", err * 100)


240857637445.16205
[[1.00000381]
 [1.00000247]
 [0.99999234]
 [1.00000069]
 [1.00000503]
 [0.99998593]
 [1.00001451]
 [0.99998805]
 [1.00001079]
 [0.9999906 ]
 [1.0000047 ]
 [1.00000256]
 [0.99999425]
 [1.00000528]
 [1.00000005]
 [0.99999878]
 [1.00000745]
 [0.99998078]
 [1.00000665]
 [1.00001741]
 [0.99999062]
 [0.99997413]
 [1.00003399]
 [0.99997383]
 [1.0000228 ]
 [1.00000769]
 [0.99996167]
 [1.00003316]
 [0.99999775]
 [0.999977  ]
 [1.00001704]
 [0.99998903]
 [1.00001514]
 [0.99999427]]
Errore soluzione LU:  0.0015974684246996228


### LU con pivotaggio a perno massimo

In [ ]:
# Vedo quanto la matrice e' mal condizionata
# così da capire come poter muovermi.
cond = np.linalg.cond(A)
print(cond)

# Le regole di partenza sono sempre le stesse viste per
# la fattorizzazione LU classica: A a rango massimo, matrice
# di permutazione P da ricercare e soddisfare il sistema
# lineare che trova la soluzione. Quest'ultima richiesta
# può essere semplificata nel seguente modo.
#
# Uso un algoritmo in place: la fattorizzazione LU
# e' stabile in senso debole in quanto la matrice L
# viene costruita indipendentemente dalle
# caratteristiche di A mentre U dipende in maniera
# esponenziale dall’ordine della matrice.
PV, L, U = spl.lu(A)
P = PV.T
y, flag = RS.Lsolve(L, P@b)

if (flag == 0):
    sol, flag1 = RS.Usolve(U, y)

# Soluzione
print(sol)

# Di quanto ci discostiamo dalla soluzione esatta?
x_esatta = np.ones_like(b)
err = np.linalg.norm(sol - x_esatta) / np.linalg.norm(x_esatta)
print("Errore soluzione LU: ", err * 100)


### QR

In [4]:
# Se la matrice che dobbiamo analizzare non
# e' simmetrica e nemmeno definita positiva
# posso usare una fattorizzazione che è sempre
# valida: Q matrice matrice ortogonale e R
# matrice triangolare superiore non singolare
# per soddisfare il seguente sistema lineare:
# { Qz = b
# { Rx = z
#
# Vedo l'indice di condizionamento e fattorizzo
cond = np.linalg.cond(A)
print(cond)
Q, R = spl.qr(A)
n = A.shape[0]

# Qz = b
z = Q.T @ b

# Rx = z - la matrice è triangolare superiore quindi devo per forza sfruttare la sua relativa
# risoluzione,Usolve()
sol, flag = RS.Usolve(R, z)
print(sol)

# Di quanto ci discostiamo dalla soluzione esatta?
# L'errore relativo e' solitamente più piccolo
# nel caso in cui la soluzione sia calcolata con
# il metodo QR. L'algoritmo è stabile in senso forte.
x_esatta = np.ones_like(b)
err = np.linalg.norm(sol - x_esatta) / np.linalg.norm(x_esatta)
print("Errore soluzione QR: ", err * 100)


240857637445.16205
[[0.99999866]
 [1.0000074 ]
 [0.99999084]
 [1.00000206]
 [1.00000139]
 [0.99999907]
 [0.99999946]
 [1.00000128]
 [0.99999915]
 [1.0000005 ]
 [1.00000026]
 [0.99999841]
 [1.00000221]
 [0.9999984 ]
 [1.00000092]
 [1.00000004]
 [0.99999765]
 [1.00000349]
 [0.99999819]
 [0.99999853]
 [1.00000255]
 [0.99999922]
 [1.00000023]
 [0.99999887]
 [0.99999994]
 [1.00000533]
 [0.99999209]
 [1.00000569]
 [0.99999533]
 [1.00000744]
 [0.99998963]
 [1.0000077 ]
 [0.99999495]
 [1.00000153]]
Errore soluzione QR:  0.0004158978680821349


### Cholesky

In [5]:
# Il metodo si applica alle matrici simmetriche
# e definite positive dalle quali è possibile
# ottenere una matrice L triangolare inferiore
# che presenta elementi diagonali positivi tali
# per cui vale la relazione A = L * L.T.
# Il sistema lineare da soddisfare in questo caso
# e':
# { Ly = b
# { L.T x = y
#
# Vedo l'indice di condizionamento e fattorizzo
cond = np.linalg.cond(A)
print(cond)

# Genero la matrice L
L = spl.cholesky(A, lower=True)

# Risolvo il sistema
y, flag = RS.Lsolve(L, b)
sol, flag = RS.Usolve(L.T, y)
print(sol)

# Di quanto ci discostiamo dalla soluzione esatta?
# L'algoritmo è stabile in senso forte.
x_esatta = np.ones_like(b)
err = np.linalg.norm(sol - x_esatta) / np.linalg.norm(x_esatta)
print("Errore soluzione Cholesky: ", err * 100)


240857637445.16205


LinAlgError: 9-th leading minor of the array is not positive definite